In [1]:
import os
import sys
import numpy as np
from taskqueue import LocalTaskQueue
import igneous.task_creation as tc
from cloudvolume import CloudVolume

In [7]:
HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(PATH)
from utilities.FileLocationManager import FileLocationManager

In [19]:
class N2N():
    viewer = None

    def __init__(self, volume, scales, layer_type, data_type, num_channels=1, chunk_size=[64,64,64]):
        self.volume = volume
        self.scales = scales
        self.layer_type = layer_type
        self.data_type = data_type
        self.chunk_size = chunk_size
        self.precomputed_vol = volume
        self.offset = [0, 0, 0]
        self.num_channels = num_channels

    def init_volume(self, path):
        info = CloudVolume.create_new_info(
            num_channels = self.volume.shape[2] if len(self.volume.shape) > 2 else 1,
            layer_type = self.layer_type,
            data_type = self.data_type, # str(self.volume.dtype),  # Channel images might be 'uint8'
            encoding = 'raw',  # raw, jpeg, compressed_segmentation, fpzip, kempressed
            resolution = self.scales,            # Voxel scaling, units are in nanometers
            voxel_offset = self.offset,          # x,y,z offset in voxels from the origin
            chunk_size = self.chunk_size,           # units are voxels
            volume_size = self.volume.shape[:3], # e.g. a cubic millimeter dataset
        )
        self.precomputed_vol = CloudVolume(f'file://{path}', mip=0, info=info, compress=True, progress=False)
        self.precomputed_vol.commit_info()
        #self.precomputed_vol[:, :, :] = self.volume[:, :, :]

In [2]:
vol = CloudVolume('precomputed://gs://wanglab-pma/allenatlas_2017_16bit_hierarch_labels_fillmissing', parallel=True, progress=True)
image = vol[:,:,:] # Download a whole image stack into a numpy array from the cloud

Downloading:  99%|█████████▉| 113/114 [00:05<00:00, 56.26it/s]

In [25]:
OUTPUT = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/structures/allen'
filepath = os.path.join(OUTPUT, 'allen.npy')
np.save(filepath, timage)

In [23]:
timage = image.reshape(320,528,456)

In [20]:
scales = [25,25,25]
data_type = np.uint16
chunks = [64,64,1]
OUTPUT = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/structures/allen'
ng = N2N(image, scales, layer_type='segmentation', data_type=data_type, chunk_size=chunks)
ng.init_volume(OUTPUT)


In [21]:
tq = LocalTaskQueue(parallel=2)


(320, 528, 456, 456)

In [26]:
# pip install numpy tifffile cloud-volume
import numpy as np
import tifffile
from cloudvolume import CloudVolume

vol = CloudVolume(
    "s3://open-neurodata/ara_2016/sagittal_10um/nissl_10um", mip=0, use_https=True
)
image = vol[:,:,:]
# load data into numpy array
#cutout = vol[512:1024, 0:512, 512:528]

Downloading: 100%|██████████| 432/432 [04:54<00:00,  2.00it/s]

In [28]:
image.shape

(1320, 800, 1140, 1)

In [29]:
# save cutout as TIFF
OUTPUT = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/structures/allen'
filepath = os.path.join(OUTPUT, 'allen.10um.tif')
tifffile.imwrite(filepath, data=np.transpose(image))